In [10]:
import tensorflow as tf
from tensorflow import keras

In [11]:
(x_train,y_train),(x_val,y_val)=keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [12]:
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float64)/255.-0.5
    y = tf.cast(y,dtype=tf.int32)
    return x,y

In [13]:
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(200).map(preprocess).shuffle(10000)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(200).map(preprocess).shuffle(10000)

db_iter = iter(db_train)
sample=next(db_iter)
print(sample[0].shape,sample[1].shape)

(200, 32, 32, 3) (200, 1)


In [19]:
#加入参数初始化、batchnorm、dropout
model = keras.Sequential([
    keras.layers.Conv2D(32,kernel_size=[3,3],padding='same',kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(64,kernel_size=[3,3],padding='same',kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.Dropout(0.6),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    keras.layers.BatchNormalization(),
    
    
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128,activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.01),
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,activation=tf.nn.softmax,
                        kernel_initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05),bias_initializer=tf.initializers.zeros)
])

In [21]:
my_optimizer = keras.optimizers.SGD(learning_rate=0.1,decay=0.005,momentum=0.9)#学习率衰减和滑动平均
model.compile(optimizer=my_optimizer,loss=tf.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
callback = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode='auto')#提前停止
]
model.fit(db_train,epochs=50,validation_data=db_val,validation_freq=1,callbacks=callback)

Epoch 1/50
250/250 [==============================] - 37s 149ms/step - loss: 3.2504 - accuracy: 0.3975 - val_loss: 2.7906 - val_accuracy: 0.2125
Epoch 2/50
250/250 [==============================] - 17s 67ms/step - loss: 1.8368 - accuracy: 0.5394 - val_loss: 1.8254 - val_accuracy: 0.5636
Epoch 3/50
250/250 [==============================] - 17s 66ms/step - loss: 1.6691 - accuracy: 0.6106 - val_loss: 1.7730 - val_accuracy: 0.5740
Epoch 4/50
250/250 [==============================] - 16s 65ms/step - loss: 1.5910 - accuracy: 0.6442 - val_loss: 1.6326 - val_accuracy: 0.6241
Epoch 5/50
250/250 [==============================] - 17s 67ms/step - loss: 1.5134 - accuracy: 0.6808 - val_loss: 1.5644 - val_accuracy: 0.6625
Epoch 6/50
250/250 [==============================] - 17s 67ms/step - loss: 1.4493 - accuracy: 0.7096 - val_loss: 1.4255 - val_accuracy: 0.7032
Epoch 7/50
250/250 [==============================] - 16s 66ms/step - loss: 1.3985 - accuracy: 0.7279 - val_loss: 1.4612 - val_accuracy

In [22]:
model.evaluate(db_val)

50/50 [==============================] - 1s 23ms/step - loss: 1.5858 - accuracy: 0.7215


[1.5858200550079347, 0.7215]